In [ ]:
# Import important libraries

import numpy as np
import sympy as sp
import scipy.integrate as integrate

In [ ]:
def stress_at_point(x,E,nu,a,app_strain):
    E = E
    nu = nu
    a = a
    delta = np.array([[1,0,0],[0,1,0],[0,0,1]])
    lambda_ = 0
    der_epsi=1e-6

    # Solving the cubis equation of lambda
    def lambda_cubic(x,a):
        global lambda_
        sym = sp.Symbol('lambda_')
        equation = (x[0]**2)/(a[0]**2 + sym) + (x[1]**2)/(a[1]**2 + sym) + (x[2]**2)/(a[2]**2 + sym) - 1
        sol =  sp.solve(equation, sym)[0]
        if sol > 0:
            lambda_ = sol
        else:
            lambda_ = 0
    
    # Defining elliptic integrals and solving them.
    def I(x, a):
        # lambda_ = lambda_cubic(x,a)
        global lambda_

        def integrand(s):
            return 1/np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))
        inte = integrate.quad(integrand, lambda_, np.inf)[0]
        return inte*np.pi*2*a[0]*a[1]*a[2]
    
    def Ii(x,a,i):
        # lambda_ = lambda_cubic(x,a)
        global lambda_

        def integrand(s):
            return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)))
        inte = integrate.quad(integrand, lambda_, np.inf)[0]
        return inte*np.pi*2*a[0]*a[1]*a[2]
    
    def Iij(x,a,i,j):
        #lambda_ = lambda_cubic(x,a)
        global lambda_
        def integrand(s):
            return (1/(np.sqrt((a[0]**2 + s)*(a[1]**2 + s)*(a[2]**2 + s))*(a[i]**2 + s)*(a[j]**2 + s)))
        inte = integrate.quad(integrand, lambda_, np.inf)[0]
        return inte*np.pi*2*a[0]*a[1]*a[2]
    
    def get_Sijkl(x,a):
        global delta, nu
        Sijkl = np.zeros((3,3,3,3))
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    for l in range(3):
                        t1 = delta[i,j]*delta[k,l]*(2*nu*Ii(x,a,i) - Ii(x,a,k) + a[i]**2*Iij(x,a,k,i))
                        t2 = (delta[i,k]*delta[j,l] + delta[i,l]*delta[j,k])*(Ii(a[i]**2*Iij(x,a,i,j)) - Ii(x,a,j) + (1-nu)*(Ii(x,a,k) + Ii(x,a,l)))
                        Sijkl[i,j,k,l] = (t1+t2)/(8*np.pi*(1-nu))
        return Sijkl
    
    #Compute derivative of Ii wrt to j direction
    def Ii_j(x,a,i,j):
        global der_epsi, delta
        xp = x + der_epsi*delta[j]
        xm = x - der_epsi*delta[j]
        return (Ii(xp,a,i) - Ii(xm,a,i))/(2*der_epsi)

    # Compute derivative of Iij wrt to k direction
    def Iij_k(x,a,i,j,k):
        global der_epsi, delta
        xp = x + der_epsi*delta[k]
        xm = x - der_epsi*delta[k]
        return (Iij(xp,a,i,j) - Iij(xm,a,i,j))/(2*der_epsi)

    # Compute double partial derivative of Iij wrt to k and l direction
    def Iij_kl(x,a,i,j,k,l):
        global der_epsi, delta
        xp_yp = x + der_epsi*delta[k] + der_epsi*delta[l]
        xm_yp = x - der_epsi*delta[k] + der_epsi*delta[l]
        xp_ym = x + der_epsi*delta[k] - der_epsi*delta[l]
        xm_ym = x - der_epsi*delta[k] - der_epsi*delta[l]
        return (Iij(xp_yp,a,i,j) - Iij(xm_yp,a,i,j) - Iij(xp_ym,a,i,j) + Iij(xm_ym,a,i,j))/(4*der_epsi**2)

    # Compute derivative of Ii wrt to j and k direction
    def Ii_jk(x,a,i,j,k):
        global der_epsi, delta
        xp_yp = x + der_epsi*delta[j] + der_epsi*delta[k]
        xm_ym = x - der_epsi*delta[j] - der_epsi*delta[k]
        xp_ym = x + der_epsi*delta[j] - der_epsi*delta[k]
        xm_yp = x - der_epsi*delta[j] + der_epsi*delta[k]
        return (Ii(xp_yp,a,i) - Ii(xm_ym,a,i) - Ii(xp_ym,a,i) + Ii(xm_yp,a,i))/(4*der_epsi**2)
    
    def get_Dijkl(x,a,S):
        global delta, nu
        Dijkl = np.zeros((3,3,3,3))
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    for l in range(3):
                        t1 = 8*np.pi*(1-nu)*S[i,j,k,l] + 2*nu*delta[k,l]*x[i]*Ii_j(x,a,i,j)
                        t2 = (1+nu)*(delta[i,l]*x[k]*Ii_j(x,a,k,j) + delta[j,l]*x[k]*Ii_j(x,a,k,i) + delta[i,k]*x[l]*Ii_j(x,a,l,j) + delta[j,k]*x[l]*Ii_j(x,a,l,i))
                        t3 = -delta[i,j]*x[k]*(Ii_j(x,a,k,l) + a[i]**2*Iij_k(x,a,k,l,l)) - (delta[i,k]*x[j] + delta[j,k]*x[i])*(Ii_j(x,a,j,l) - a[i]**2*Iij_k(x,a,i,j,l))
                        t4 = -(delta[i,l]*x[j] + delta[j,l]*x[i])*(Ii_j(x,a,j,k) - a[i]**2*Iij_k(x,a,i,j,k)) - x[i]*x[j]*(Ii_jk(x,a,j,l,k) - a[i]**2*Iij_kl(x,a,i,j,l,k))
                        Dijkl[i,j,k,l] = (t1+t2+t3+t4)/(8*np.pi*(1-nu))
        return Dijkl
    
    # Calculate epsilon tensor
    def get_epsilon(D,app_strain):
        epsilon = np.zeros((3,3))
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    for l in range(3):
                        epsilon[i,j] = epsilon[i,j] + D[i,j,k,l]*app_strain[k,l]
        return epsilon
    
    # Calculate stress tensor for 3D
    def get_stress(epsilon,E,nu):
        sig11 = E/(1-nu**2)*(epsilon[0,0] + nu*epsilon[1,1] + nu*epsilon[2,2])
        sig22 = E/(1-nu**2)*(nu*epsilon[0,0] + epsilon[1,1] + nu*epsilon[2,2])
        sig33 = E/(1-nu**2)*(nu*epsilon[0,0] + nu*epsilon[1,1] + epsilon[2,2])
        sig12 = E/(1+nu)*epsilon[0,1]
        sig13 = E/(1+nu)*epsilon[0,2]
        sig23 = E/(1+nu)*epsilon[1,2]
        return np.array([[sig11,sig12,sig13],[sig12,sig22,sig23],[sig13,sig23,sig33]])
    
    lambda_ = lambda_cubic(x, a)
    S = get_Sijkl(x, a)
    D = get_Dijkl(x, a, S)
    epsilon = get_epsilon(D, app_strain)
    stress = get_stress(epsilon, E, nu)
    return stress
